Implementing this code here: https://github.com/trevorhastie/glmnet/blob/3b268cebc7a04ff0c7b22931cb42b4c328ede307/R/glmnetFlex.R#L217C1-L233C6

'''
# standardize x if necessary
    if (intercept) {
        xm <- meansd$mean
    } else {
        xm <- rep(0.0, times = nvars)
    }
    if (standardize) {
        xs <- meansd$sd
    } else {
        xs <- rep(1.0, times = nvars)
    }
    if (!inherits(x, "sparseMatrix")) {
        x <- scale(x, xm, xs)
    }
'''

In [2]:
import numpy as np
from glmnet.base import Design
from glmnet import GLMNet
import rpy2
%load_ext rpy2.ipython

n, p = 100, 50
rng = np.random.default_rng(0)
X = rng.standard_normal((n, p))
Y = rng.standard_normal(n)
beta = rng.standard_normal(p)
W = rng.uniform(1, 2, size=n)
W /= W.mean()
%R -i X,Y

# Without weights (all 1's)

## `standardize=True`, `intercept=True`

In [5]:
standardize, intercept = True, True
W = np.ones(n)
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * (Y - Y.mean()))).max() / n)
G.lambda_max_

0.2820689073452428

In [6]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize)
print(max(G$lambda)*nrow(X))

[1] 28.20689


Loading required package: Matrix
Loaded glmnet 4.1-7


## `standardize=True`, `intercept=False`

In [7]:
standardize, intercept = True, False
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * Y)).max()/n)
G.lambda_max_

0.2910602370580807

In [8]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize)
print(max(G$lambda)*nrow(X))

[1] 29.10602


## `standardize=False`, `intercept=True`

In [9]:
standardize, intercept = False, True
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * (Y - Y.mean()))).max()/n)
G.lambda_max_

0.27946460564487424

In [10]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize)
print(max(G$lambda)*nrow(X))

[1] 27.94646


## `standardize=False`, `intercept=False`

In [11]:
standardize, intercept = False, False
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * Y)).max()/n)
G.lambda_max_

0.2883729197021402

In [12]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize)
print(max(G$lambda)*nrow(X))

[1] 28.83729


# With weights

## `standardize=True`, `intercept=True`

In [13]:
standardize, intercept = True, True
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, W, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y, sample_weight=W)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * (Y - Y.mean()))).max()/n)
G.lambda_max_

0.2820689073452428

In [14]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize, weights=W)
print(max(G$lambda)*nrow(X))

[1] 28.20689


## `standardize=True`, `intercept=False`

In [15]:
standardize, intercept = True, False
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, W, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y, sample_weight=W)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * Y)).max()/n)
G.lambda_max_

0.2910602370580807

In [16]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize, weights=W)
print(max(G$lambda)*nrow(X))

[1] 29.10602


## `standardize=False`, `intercept=True`

In [32]:
standardize, intercept = False, True
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, W, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNet(lambda_values=np.linspace(0.05,1,51)[::-1],
           fit_intercept=intercept, standardize=standardize)
G.fit(X, Y, sample_weight=W)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * (Y - Y.mean()))).max()/n)
G.lambda_max_

27.946460564487424

In [33]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize, weights=W)
print(max(G$lambda)*nrow(X))

[1] 27.94646


## `standardize=False`, `intercept=False`

In [34]:
standardize, intercept = False, False
xm = (X * W[:,None]).sum(0) / W.sum()
x2 = (X**2 * W[:,None]).sum(0) / W.sum()
xs = np.sqrt(x2 - xm**2)
if not standardize:
    xs = np.ones_like(xs)
if not intercept:
    xm = np.zeros_like(xm)
X_ = (X - xm[None,:]) / xs[None,:]
D = Design(X, W, intercept=intercept, standardize=standardize)
assert np.allclose(D.centers_, xm)
assert np.allclose(D.scaling_, xs)
assert np.allclose(D @ np.hstack([0, beta]), X_ @ beta)
G = GLMNetPath(lambda_values=np.linspace(0.05,1,51)[::-1],
               fit_intercept=intercept, standardize=standardize)
G.fit(X, Y, sample_weight=W)
assert np.allclose(G.lambda_max_, np.fabs(X_.T @ (W * Y)).max())
G.lambda_max_

28.83729197021401

In [35]:
%%R -i standardize,intercept,W
library(glmnet)
G=glmnet(X, Y, intercept=intercept, standardize=standardize, weights=W)
print(max(G$lambda)*nrow(X))

[1] 28.83729
